In [3]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import os
import json

display(HTML('<h1 style="color:#BD052D">FURIA Fan Insights</h1>'))
display(HTML('<h2>Conhecendo o Fã</h2>'))

display(HTML('<h3>Informações Básicas</h3>'))

name = widgets.Text(description="Nome:", style={'description_width': '120px'})
email = widgets.Text(description="Email:", style={'description_width': '120px'})
cpf = widgets.Text(description="CPF:", style={'description_width': '120px'})
birth_date = widgets.DatePicker(description="Data de Nascimento:", style={'description_width': '120px'})
city = widgets.Text(description="Cidade:", style={'description_width': '120px'})
state = widgets.Dropdown(
    options=[
        'AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA',
        'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN',
        'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO'
    ],
    description='Estado:',
    style={'description_width': '120px'}
)

display(name, email, cpf, birth_date, city, state)

Text(value='', description='Nome:', style=TextStyle(description_width='120px'))

Text(value='', description='Email:', style=TextStyle(description_width='120px'))

Text(value='', description='CPF:', style=TextStyle(description_width='120px'))

DatePicker(value=None, description='Data de Nascimento:', step=1, style=DescriptionStyle(description_width='12…

Text(value='', description='Cidade:', style=TextStyle(description_width='120px'))

Dropdown(description='Estado:', options=('AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS…

In [4]:
display(HTML('<h3>Interesses em E-sports</h3>'))

games = widgets.SelectMultiple(
    options=['CS:GO', 'CS2', 'League of Legends', 'Valorant', 'Rainbow Six', 'Dota 2', 'Fortnite', 'Apex Legends', 'PUBG', 'Overwatch', 'Rocket League', 'FIFA'],
    description='Jogos:',
    rows=6,
    style={'description_width': '120px'}
)

furia_teams = widgets.SelectMultiple(
    options=['CS:GO/CS2', 'League of Legends', 'Valorant', 'Apex Legends', 'Rainbow Six', 'Rocket League', 'PUBG'],
    description='Times FURIA:',
    rows=5,
    style={'description_width': '120px'}
)

fan_activities = widgets.SelectMultiple(
    options=['Assisto transmissões online', 'Vou a eventos presenciais', 'Compro produtos oficiais', 
             'Participo de comunidades online', 'Sigo jogadores nas redes sociais', 'Jogo os mesmos jogos'],
    description='Atividades:',
    rows=4,
    style={'description_width': '120px'}
)

events_attended = widgets.Text(
    description='Eventos (último ano):',
    placeholder='Ex: IEM Rio Major, BLAST Premier, etc.',
    style={'description_width': '120px'}
)

products = widgets.SelectMultiple(
    options=['Camisetas', 'Moletons', 'Bonés', 'Mousepad', 'Cadeira Gamer', 'Periféricos', 'Outros'],
    description='Produtos FURIA:',
    rows=4,
    style={'description_width': '120px'}
)

display(games, furia_teams, fan_activities, events_attended, products)

SelectMultiple(description='Jogos:', options=('CS:GO', 'CS2', 'League of Legends', 'Valorant', 'Rainbow Six', …

SelectMultiple(description='Times FURIA:', options=('CS:GO/CS2', 'League of Legends', 'Valorant', 'Apex Legend…

SelectMultiple(description='Atividades:', options=('Assisto transmissões online', 'Vou a eventos presenciais',…

Text(value='', description='Eventos (último ano):', placeholder='Ex: IEM Rio Major, BLAST Premier, etc.', styl…

SelectMultiple(description='Produtos FURIA:', options=('Camisetas', 'Moletons', 'Bonés', 'Mousepad', 'Cadeira …

In [5]:
# Seção de redes sociais
display(HTML('<h3>Redes Sociais</h3>'))

# Perfis de redes sociais
twitter = widgets.Text(description='Twitter/X:', placeholder='@username', style={'description_width': '120px'})
instagram = widgets.Text(description='Instagram:', placeholder='@username', style={'description_width': '120px'})
facebook = widgets.Text(description='Facebook:', placeholder='URL do perfil', style={'description_width': '120px'})
twitch = widgets.Text(description='Twitch:', placeholder='username', style={'description_width': '120px'})
youtube = widgets.Text(description='YouTube:', placeholder='URL do canal', style={'description_width': '120px'})
discord = widgets.Text(description='Discord:', placeholder='username#0000', style={'description_width': '120px'})

# Plataformas de e-sports
faceit = widgets.Text(description='FACEIT:', placeholder='username ou URL', style={'description_width': '120px'})
gamersclub = widgets.Text(description='GamersClub:', placeholder='username ou URL', style={'description_width': '120px'})
esea = widgets.Text(description='ESEA:', placeholder='username ou URL', style={'description_width': '120px'})

# Exibir widgets de redes sociais
display(twitter, instagram, facebook, twitch, youtube, discord)
display(faceit, gamersclub, esea)

Text(value='', description='Twitter/X:', placeholder='@username', style=TextStyle(description_width='120px'))

Text(value='', description='Instagram:', placeholder='@username', style=TextStyle(description_width='120px'))

Text(value='', description='Facebook:', placeholder='URL do perfil', style=TextStyle(description_width='120px'…

Text(value='', description='Twitch:', placeholder='username', style=TextStyle(description_width='120px'))

Text(value='', description='YouTube:', placeholder='URL do canal', style=TextStyle(description_width='120px'))

Text(value='', description='Discord:', placeholder='username#0000', style=TextStyle(description_width='120px')…

Text(value='', description='FACEIT:', placeholder='username ou URL', style=TextStyle(description_width='120px'…

Text(value='', description='GamersClub:', placeholder='username ou URL', style=TextStyle(description_width='12…

Text(value='', description='ESEA:', placeholder='username ou URL', style=TextStyle(description_width='120px'))

In [6]:
# Upload de documento e finalização
display(HTML('<h3>Upload de Documento e Finalização</h3>'))

# Upload de documento (simulado)
file_upload = widgets.FileUpload(
    accept='.jpg,.jpeg,.png,.pdf',
    multiple=False,
    description='Upload ID:',
    style={'description_width': '120px'}
)
display(file_upload)

# Comentários adicionais
comments = widgets.Textarea(
    placeholder='Conte-nos mais sobre sua experiência como fã da FURIA...',
    description='Comentários:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='80%', height='100px')
)
display(comments)

# Botão de envio
submit_button = widgets.Button(
    description='Enviar Informações',
    button_style='success',
    icon='check'
)

output = widgets.Output()

def save_data(b):
    with output:
        # Criar um diretório de dados se não existir
        os.makedirs('data', exist_ok=True)
        
        # Coletar dados de todos os widgets
        fan_data = {
            'basic_info': {
                'name': name.value,
                'email': email.value,
                'cpf': cpf.value,
                'birth_date': str(birth_date.value) if birth_date.value else '',
                'city': city.value,
                'state': state.value
            },
            'esports_interests': {
                'games': list(games.value),
                'furia_teams': list(furia_teams.value),
                'fan_activities': list(fan_activities.value),
                'events_attended': events_attended.value,
                'products': list(products.value)
            },
            'social_media': {
                'twitter': twitter.value,
                'instagram': instagram.value,
                'facebook': facebook.value,
                'twitch': twitch.value,
                'youtube': youtube.value,
                'discord': discord.value,
                'faceit': faceit.value,
                'gamersclub': gamersclub.value,
                'esea': esea.value
            },
            'comments': comments.value
        }
        
        # Lidar com o upload de documento (se houver)
        if file_upload.value:
            # Obter o primeiro arquivo carregado
            uploaded_file = next(iter(file_upload.value.values()))
            filename = uploaded_file['metadata']['name']
            content = uploaded_file['content']
            
            # Salvar o arquivo
            with open(f'data/{filename}', 'wb') as f:
                f.write(content)
            fan_data['document'] = f'data/{filename}'
        
        # Salvar os dados em um arquivo JSON
        with open('data/fan_profile.json', 'w') as f:
            json.dump(fan_data, f, indent=4)
        
        print("✅ Dados salvos com sucesso em 'data/fan_profile.json'!")
        print("✅ Obrigado por compartilhar suas informações como fã da FURIA!")
        
        # Visualização básica dos dados coletados
        print("\n📊 Resumo dos dados coletados:")
        print(f"Nome: {fan_data['basic_info']['name']}")
        if fan_data['esports_interests']['furia_teams']:
            print(f"Times FURIA seguidos: {', '.join(fan_data['esports_interests']['furia_teams'])}")
        if fan_data['esports_interests']['games']:
            print(f"Jogos de interesse: {', '.join(fan_data['esports_interests']['games'])}")
        if fan_data['esports_interests']['products']:
            print(f"Produtos FURIA: {', '.join(fan_data['esports_interests']['products'])}")

# Anexar o manipulador de eventos ao botão
submit_button.on_click(save_data)

# Exibir o botão e saída
display(submit_button, output)

FileUpload(value=(), accept='.jpg,.jpeg,.png,.pdf', description='Upload ID:')

Textarea(value='', description='Comentários:', layout=Layout(height='100px', width='80%'), placeholder='Conte-…

Button(button_style='success', description='Enviar Informações', icon='check', style=ButtonStyle())

Output()

In [7]:
# Simulação de análise de dados com IA
display(HTML('<h3>Análise do Perfil de Fã</h3>'))

def analyze_fan_profile(b):
    try:
        with open('data/fan_profile.json', 'r') as f:
            data = json.load(f)
        
        with viz_output:
            viz_output.clear_output()
            
            # Simular análise de perfil com IA
            print("🔍 Analisando perfil de fã...")
            import time
            time.sleep(1.5)
            
            print("\n✅ Validação de identidade concluída")
            print("✅ Perfil de redes sociais analisado")
            print("✅ Perfil de plataformas de e-sports validado")
            
            print("\n📊 Insights do Fã:")
            print(f"👤 {data['basic_info']['name']} de {data['basic_info']['city']}-{data['basic_info']['state']}")
            
            if data['esports_interests']['furia_teams']:
                primary_team = list(data['esports_interests']['furia_teams'])[0]
                print(f"🎮 Fã principal do time de {primary_team} da FURIA")
            
            if data['esports_interests']['games']:
                print(f"🎲 Principais jogos: {', '.join(list(data['esports_interests']['games'])[:3])}")
            
            # Gerar recomendações personalizadas
            print("\n🎯 Recomendações personalizadas:")
            
            if data['esports_interests']['furia_teams'] and 'CS:GO/CS2' in data['esports_interests']['furia_teams']:
                print("📺 Nova série de conteúdo sobre os bastidores do time de CS da FURIA")
            
            if 'Compro produtos oficiais' in data['esports_interests']['fan_activities']:
                print("🛒 Nova coleção de produtos FURIA disponível com 10% de desconto para fãs registrados")
            
            if 'Vou a eventos presenciais' in data['esports_interests']['fan_activities']:
                print("🎫 Pré-venda exclusiva para o próximo evento da FURIA em São Paulo")
            
            print("\n🏆 Seu nível de engajamento: Hardcore Fan")
            
            # Criar visualização simples
            display(HTML("<hr>"))
            display(HTML("<h4>Mapa de Interesses</h4>"))
            
            # Gerar uma visualização básica com pandas
            if data['esports_interests']['games']:
                import matplotlib.pyplot as plt
                import numpy as np
                
                # Criar dados para visualização
                games = list(data['esports_interests']['games'])
                values = np.random.randint(70, 100, len(games))
                
                fig, ax = plt.subplots(figsize=(10, 6))
                bars = ax.bar(games, values, color='#BD052D')
                ax.set_title('Nível de Interesse por Jogo')
                ax.set_xlabel('Jogos')
                ax.set_ylabel('Interesse (0-100)')
                ax.set_ylim(0, 100)
                
                # Adicionar valores nas barras
                for bar in bars:
                    height = bar.get_height()
                    ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                            f'{height}%', ha='center', va='bottom')
                
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.show()
                
    except FileNotFoundError:
        with viz_output:
            viz_output.clear_output()
            print("⚠️ Nenhum dado encontrado. Por favor, preencha e envie o formulário primeiro.")

analyze_button = widgets.Button(
    description='Analisar Perfil',
    button_style='info',
    icon='search'
)

viz_output = widgets.Output()

analyze_button.on_click(analyze_fan_profile)

display(analyze_button, viz_output)

Button(button_style='info', description='Analisar Perfil', icon='search', style=ButtonStyle())

Output()